#### Topics:

    BEGIN…END – create a statement block that consists of multiple Transact-SQL statements that execute together.
    IF ELSE – execute a statement block based on a condition.
    WHILE – repeatedly execute a set of statements based on a condition as long as the condition is true.
    BREAK – exit the loop immediately and skip the rest of the code after it within a loop.
    CONTINUE – skip the current iteration of the loop immediately and continue the next one.

### BEGIN...END Statement

The BEGIN...END statement is used to define a statement block. A statement block consists of a set of SQL statements that execute together. A statement block is also known as a batch.

In other words, if statements are sentences, the BEGIN...END statement allows you to define paragraphs.

The following illustrates the syntax of the BEGIN...END statement:

    BEGIN
        { sql_statement | statement_block}
    END

In [19]:
import pyodbc
import os
import pandas as pd

#Check if drivers are installed
#[x for x in pyodbc.drivers() if x.startswith("Microsoft Access Driver")]

# Define the connection string
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost;'
    r'DATABASE=BikeStores;'
    r'Trusted_Connection=yes;'
)

# Establish the connection
conn = pyodbc.connect(conn_str, autocommit=True)

# Create a cursor
cursor = conn.cursor()

#### 1. Using begin end

In [5]:

cursor.execute('''
BEGIN
    SELECT
        product_id,
        product_name, list_price
    FROM
        production.products
    WHERE
        list_price > 100;

    IF @@ROWCOUNT = 0
        PRINT 'No product with price greater than 100000 found';
END

''')


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_id,product_name,list_price
0,1,Trek 820 - 2016,379.99
1,2,Ritchey Timberwolf Frameset - 2016,749.99
2,3,Surly Wednesday Frameset - 2016,999.99
3,4,Trek Fuel EX 8 29 - 2016,2899.99
4,5,Heller Shagamaw Frame - 2016,1320.99
5,6,Surly Ice Cream Truck Frameset - 2016,469.99
6,7,Trek Slash 8 27.5 - 2016,3999.99
7,8,Trek Remedy 29 Carbon Frameset - 2016,1799.99
8,9,Trek Conduit+ - 2016,2999.99
9,10,Surly Straggler - 2016,1549.00


#### 2. Nesting begin End

In [7]:

#Now, you can put everything together and execute the following code block to get
#a list of products whose model year is 2018:

cursor.execute('''
BEGIN
    DECLARE @name VARCHAR(MAX);

    SELECT TOP 1
        @name = product_name
    FROM
        production.products
    ORDER BY
        list_price DESC;
    
    IF @@ROWCOUNT <> 0
    BEGIN
        PRINT 'The most expensive product is ' + @name
    END
    ELSE
    BEGIN
        PRINT 'No product found';
    END;
END
''')


### IF ELSE

The IF...ELSE statement is a control-flow statement that allows you to execute or skip a statement block based on a specified condition.

The following illustrates the syntax of the IF statement:

    IF boolean_expression   
    BEGIN
        { statement_block }
    END

The following illustrates the syntax of the IF ELSE statement:

    IF Boolean_expression
    BEGIN
        -- Statement block executes when the Boolean expression is TRUE
    END
    ELSE
    BEGIN
        -- Statement block executes when the Boolean expression is FALSE
    END

In [11]:
cursor.execute('''
BEGIN
    DECLARE @sales INT;

    SELECT 
        @sales = SUM(list_price * quantity)
    FROM
        sales.order_items i
        INNER JOIN sales.orders o ON o.order_id = i.order_id
    WHERE
        YEAR(order_date) = 2018;

    SELECT @sales;

    IF @sales > 1000000
    BEGIN
        PRINT 'Great! The sales amount in 2018 is greater than 1,000,000';
    END
END
''')



# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]


# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,
0,2023989


If ELSE

In [12]:
cursor.execute('''
BEGIN
    DECLARE @sales INT;

    SELECT 
        @sales = SUM(list_price * quantity)
    FROM
        sales.order_items i
        INNER JOIN sales.orders o ON o.order_id = i.order_id
    WHERE
        YEAR(order_date) = 2017;

    SELECT @sales;

    IF @sales > 10000000
    BEGIN
        PRINT 'Great! The sales amount in 2018 is greater than 10,000,000';
    END
    ELSE
    BEGIN
        PRINT 'Sales amount in 2017 did not reach 10,000,000';
    END
END

''')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,
0,3845515


### WHILE

The WHILE statement is a control-flow statement that allows you to execute a statement block repeatedly as long as a specified condition is TRUE.

The following illustrates the syntax of the WHILE statement:

    WHILE Boolean_expression   
         { sql_statement | statement_block}  

In this syntax:

First, the Boolean_expression is an expression that evaluates to TRUE or FALSE.

Second, sql_statement | statement_block is any Transact-SQL statement or a set of Transact-SQL statements. A statement block is defined using the BEGIN...END statement.

If the Boolean_expression evaluates to FALSE when entering the loop, no statement inside the WHILE loop will be executed.

Inside the WHILE loop, you must change some variables to make the Boolean_expression returns FALSE at some points. Otherwise, you will have an indefinite loop.

Note that if the Boolean_expression contains a SELECT statement, it must be enclosed in parentheses.

To exit the current iteration of the loop immediately, you use the BREAK statement. To skip the current iteration of the loop and start the new one, you use the CONTINUE statement.

In [21]:

# T-SQL query
tsql_query = """
-- Create a temporary table to store results
CREATE TABLE TempResult (
    Counter INT
);

DECLARE @Counter INT;
SET @Counter = 1;

-- WHILE loop
WHILE @Counter <= 10
BEGIN
    INSERT INTO TempResult (Counter)
    VALUES (@Counter);
    
    SET @Counter = @Counter + 1;
END;

-- Select the results
SELECT * FROM TempResult;

"""

# Execute the T-SQL query
cursor.execute(tsql_query)
conn.commit()

# Fetch the results
select_query = "SELECT * FROM TempResult;"
cursor.execute(select_query)
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,Counter
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


### WHILE... BREAK

In [22]:
cursor.execute('''
DECLARE @counter INT = 0;

WHILE @counter <= 5
BEGIN
    SET @counter = @counter + 1;
    IF @counter = 4
        BREAK;
    PRINT @counter;
END

''')

# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

ProgrammingError: No results.  Previous SQL was not a query.

### CONTINUE

The CONTINUE statement stops the current iteration of the loop and starts the new one. The following illustrates the syntax of the CONTINUE statement:

    WHILE Boolean_expression
    BEGIN
        -- code to be executed
        IF condition
            CONTINUE;
        -- code will be skipped if the condition is met
    END

In this syntax, the current iteration of the loop is stopped once the condition evaluates to TRUE. The next iteration of the loop will continue until the Boolean_expression evaluates to FALSE.

In [23]:
cursor.execute('''
DECLARE @counter INT = 0;

WHILE @counter < 5
BEGIN
    SET @counter = @counter + 1;
    IF @counter = 3
        CONTINUE;	
    PRINT @counter;
END

''')

In this example:

First, we declared a variable named @counter and set its value to zero.
Then, the WHILE loop started. Inside the WHILE loop, we increased the counter by one in each iteration. If the @counter was three, we skipped printing out the value using the CONTINUE statement. That’s why in the output, you do not see the number three is showing up.